# 1º Modelagem e Separação dos dados necessários:

### Vamos utilizar 3 arquivos para pegar as informações necessárias:

### BPA (Balanço Patrimonial Ativo)
### BPP (Balanço Patrimonial Passivo)
### DRE (Demonstração de Resultado)

### Vamos trabalhar com as seguintes empresas:

### MGLU3, PETR4, WEGE4, ELET3, VALE3

### Periodo máximo é 2011

### anotações:

-Temos que utilizar o arquivo ind, porque ele tem os indicadores individuais e não consolidado
-4 trimestre é o fechamento anual

# 1 Empilhar os dados por Trimestre, ano e tipo de arquivo

### 1.1 Empilhar os dados por Trimestre DRE

In [1]:
import pandas as pd
import numpy as np

In [2]:
DRE=pd.DataFrame()
for x in range(11,23):
    df=pd.read_csv(fr"Dados_Brutos\itr_cia_aberta_DRE_ind_20{x}.csv",delimiter=';',encoding="latin-1",decimal='.')
    df=df[['DENOM_CIA','CNPJ_CIA','CD_CONTA','DT_INI_EXERC','DT_REFER','DT_FIM_EXERC','ORDEM_EXERC','DS_CONTA','VL_CONTA','ST_CONTA_FIXA']]
    df['year1'] = pd.DatetimeIndex(df['DT_REFER']).year
    df['month1'] = pd.DatetimeIndex(df['DT_REFER']).month
    ano=int(f"20{x}")
    df=df[(df["year1"]==ano)].reset_index(drop=True)
    df=df[(df["month1"]==3)|(df["month1"]==6)|(df["month1"]==9)|(df["month1"]==12)].reset_index(drop=True)
    df['year1'] = pd.DatetimeIndex(df['DT_FIM_EXERC']).year
    df['month1'] = pd.DatetimeIndex(df['DT_FIM_EXERC']).month
    df['year2'] = pd.DatetimeIndex(df['DT_INI_EXERC']).year
    df['month2'] = pd.DatetimeIndex(df['DT_INI_EXERC']).month
    df=df[(df["month1"]==3)&(df["month2"]==1)|(df["month1"]==6)&(df["month2"]==4)|(df["month1"]==9)&(df["month2"]==7)|(df["month1"]==12)&(df["month2"]==10)].reset_index(drop=True)
    df=df[(df["ST_CONTA_FIXA"]=="S")&(df["ORDEM_EXERC"]=="ÚLTIMO")].reset_index(drop=True)
    df["Nome Empresa"]=df['DENOM_CIA']
    df=df[['Nome Empresa','CNPJ_CIA','DT_REFER','DS_CONTA','CD_CONTA','VL_CONTA']]
    DRE=pd.concat([df,DRE]).reset_index(drop=True)
    #DRE.to_csv(rf"C:\Users\marqu\Desktop\Estudos Finanças\Cuspidão\DRE.csv",decimal=',',sep=';', encoding='latin-1',index=False)

### 1.2 Empilhar os dados por Trimestre BPA

In [3]:
BPA=pd.DataFrame()
for x in range(11,23):
    df=pd.read_csv(fr"Dados_Brutos\itr_cia_aberta_BPA_ind_20{x}.csv",delimiter=';',encoding="latin-1",decimal='.')
    df=df[['DENOM_CIA','CNPJ_CIA','CD_CONTA','DT_REFER','DT_FIM_EXERC','ORDEM_EXERC','DS_CONTA','VL_CONTA','ST_CONTA_FIXA']]
    ano=int(f"20{x}")
    df['year1'] = pd.DatetimeIndex(df['DT_FIM_EXERC']).year
    df['month1'] = pd.DatetimeIndex(df['DT_FIM_EXERC']).month
    df=df[(df["year1"]==ano)].reset_index(drop=True)
    df=df[(df["month1"]==3)|(df["month1"]==6)|(df["month1"]==9)|(df["month1"]==12)].reset_index(drop=True)
    df=df[(df["ST_CONTA_FIXA"]=="S")&(df["ORDEM_EXERC"]=="ÚLTIMO")].reset_index(drop=True)
    df["Nome Empresa"]=df['DENOM_CIA']
    df=df[['Nome Empresa','CNPJ_CIA','DT_REFER','DS_CONTA','CD_CONTA','VL_CONTA']]
    BPA=pd.concat([df,BPA]).reset_index(drop=True)

### 1.3 Empilhar os dados por Trimestre BPP

In [4]:
BPP=pd.DataFrame()
for x in range(11,23):
    df=pd.read_csv(fr"Dados_Brutos\itr_cia_aberta_BPP_ind_20{x}.csv",delimiter=';',encoding="latin-1",decimal='.')
    df=df[['DENOM_CIA','CNPJ_CIA','CD_CONTA','DT_REFER','DT_FIM_EXERC','ORDEM_EXERC','DS_CONTA','VL_CONTA','ST_CONTA_FIXA']]
    ano=int(f"20{x}")
    df['year1'] = pd.DatetimeIndex(df['DT_FIM_EXERC']).year
    df['month1'] = pd.DatetimeIndex(df['DT_FIM_EXERC']).month
    df=df[(df["year1"]==ano)].reset_index(drop=True)
    df=df[(df["month1"]==3)|(df["month1"]==6)|(df["month1"]==9)|(df["month1"]==12)].reset_index(drop=True)
    df=df[(df["ST_CONTA_FIXA"]=="S")&(df["ORDEM_EXERC"]=="ÚLTIMO")].reset_index(drop=True)
    df["Nome Empresa"]=df['DENOM_CIA']
    df=df[['Nome Empresa','CNPJ_CIA','DT_REFER','DS_CONTA','CD_CONTA','VL_CONTA']]
    BPP=pd.concat([df,BPP]).reset_index(drop=True)

# 2 Separar só os indicadores necessários

### 2.1 Separar só os indicadores necessários DRE,BPA e BPP

In [5]:
Completao=pd.concat([BPP,BPA,DRE]).reset_index(drop=True)
Completao['DS_CONTA']=Completao['DS_CONTA'].str.replace('Reservas de Lucros', 'Reservas de Lucro', regex=True)
Completao['DS_CONTA']=Completao['DS_CONTA'].str.replace('Obrigações sociais e trabalhistas', 'Obrigações Sociais e Trabalhistas', regex=True)
Completao['DS_CONTA']=Completao['DS_CONTA'].str.replace('Outras Obrigações Sociais e Trabalhistas', 'Obrigações Sociais e Trabalhistas', regex=True)


In [6]:
Completao=Completao[(Completao['DS_CONTA']=='Outras Despesas/Receitas Operacionais')|
                    (Completao['DS_CONTA']=='Despesas/Receitas Operacionais')|
                    (Completao['DS_CONTA']=='Receitas de Prestação de Serviços')|
                    (Completao['DS_CONTA']=='Receita de Venda de Bens e/ou Serviços')|
                    (Completao['DS_CONTA']=='Lucro/Prejuízo do Período')|
                    (Completao['DS_CONTA']=='Ativo Total')|
                    (Completao['DS_CONTA']=='Ativo Circulante')|
                    (Completao['DS_CONTA']=='Contas a Receber')|
                    (Completao['DS_CONTA']=='Passivo Total')|
                    (Completao['DS_CONTA']=='Passivo Circulante')|
                    (Completao['DS_CONTA']=='Obrigações Sociais e Trabalhistas')|
                    (Completao['DS_CONTA']=='Obrigações Fiscais')|
                    (Completao['DS_CONTA']=='Reservas de Lucro')|
                    (Completao['DS_CONTA']=='Patrimônio Líquido')].reset_index(drop=True)
Completao2=Completao

### 2.2 Informações Cadastrais

In [7]:
cad=pd.read_csv(fr"Depara\cad.csv",delimiter=';',encoding="latin-1",decimal='.')
cad=cad[['DENOM_SOCIAL','SIT','SETOR_ATIV','TP_MERC','CNPJ_CIA']]

In [8]:
tick=pd.read_excel(fr"Depara\tick.xlsx")

In [9]:
DEpara = pd.merge( tick, cad, left_on='CNPJ', right_on='CNPJ_CIA')
DEpara=DEpara[(DEpara['SIT']=='ATIVO') &(DEpara['TP_MERC']=='BOLSA')].reset_index(drop=True)
DEpara=DEpara.drop_duplicates(subset="Ticker")
DEpara = DEpara.drop(columns=['CNPJ'])

In [10]:
Completao2 = pd.merge( Completao2, DEpara, left_on='CNPJ_CIA', right_on='CNPJ_CIA')
Completao2 = Completao2.drop(columns=['CNPJ_CIA','Razão Social','DENOM_SOCIAL','SIT','TP_MERC'])

In [11]:
Completao2['Ticker']=Completao2['Ticker'].apply(str).str[0:4]
Completao2=Completao2.drop_duplicates(subset=['Nome Empresa','Ticker','SETOR_ATIV','DT_REFER','DS_CONTA','VL_CONTA'])

### 2.2 Colunar os indicadores

In [12]:
table = pd.pivot_table(Completao2, values='VL_CONTA', index=['Nome Empresa','Ticker','SETOR_ATIV', 'DT_REFER'],columns=['DS_CONTA'], aggfunc=np.sum, fill_value=0)
table=table.reset_index()

In [13]:
EMPRESAS=table['Ticker'].values
EMPRESAS= list(dict.fromkeys(EMPRESAS))
table2=pd.DataFrame()
for x in EMPRESAS:
    
    row=table[table["Ticker"]==x].reset_index(drop=True)
    row["Receita"]=np.where(row['Receita de Venda de Bens e/ou Serviços']==0,row['Despesas/Receitas Operacionais'],row['Receita de Venda de Bens e/ou Serviços'])
    row["Receita"]=np.where(row['Receita']==0,row['Outras Despesas/Receitas Operacionais'],row['Receita'])
    row["Receita"]=np.where(row['Receita']==0,row['Receitas de Prestação de Serviços'],row['Receita'])

                                                 
    if x=='ITUB':
        
        print(row[row["Ticker"]=='ITUB'])
    
    table2=pd.concat([table2,row]).reset_index(drop=True)

DS_CONTA                Nome Empresa Ticker SETOR_ATIV    DT_REFER  \
0         ITAU UNIBANCO HOLDING S.A.   ITUB     Bancos  2011-03-31   
1         ITAU UNIBANCO HOLDING S.A.   ITUB     Bancos  2011-06-30   
2         ITAU UNIBANCO HOLDING S.A.   ITUB     Bancos  2011-09-30   
3         ITAU UNIBANCO HOLDING S.A.   ITUB     Bancos  2012-03-31   
4         ITAU UNIBANCO HOLDING S.A.   ITUB     Bancos  2012-06-30   
5         ITAU UNIBANCO HOLDING S.A.   ITUB     Bancos  2012-09-30   
6         ITAU UNIBANCO HOLDING S.A.   ITUB     Bancos  2013-03-31   
7         ITAU UNIBANCO HOLDING S.A.   ITUB     Bancos  2013-06-30   
8         ITAU UNIBANCO HOLDING S.A.   ITUB     Bancos  2013-09-30   
9         ITAU UNIBANCO HOLDING S.A.   ITUB     Bancos  2014-03-31   
10        ITAU UNIBANCO HOLDING S.A.   ITUB     Bancos  2014-06-30   
11        ITAU UNIBANCO HOLDING S.A.   ITUB     Bancos  2014-09-30   
12        ITAU UNIBANCO HOLDING S.A.   ITUB     Bancos  2015-03-31   
13        ITAU UNIBA

In [14]:
table2 = table2.drop(columns=['Receita de Venda de Bens e/ou Serviços', 'Despesas/Receitas Operacionais','Despesas/Receitas Operacionais','Receitas de Prestação de Serviços','Outras Despesas/Receitas Operacionais'])


In [15]:
print(table2.head())

DS_CONTA                  Nome Empresa Ticker      SETOR_ATIV    DT_REFER  \
0         3R PETROLEUM ÓLEO E GÁS S.A.   RRRP  Petróleo e Gás  2020-03-31   
1         3R PETROLEUM ÓLEO E GÁS S.A.   RRRP  Petróleo e Gás  2020-06-30   
2         3R PETROLEUM ÓLEO E GÁS S.A.   RRRP  Petróleo e Gás  2020-09-30   
3         3R PETROLEUM ÓLEO E GÁS S.A.   RRRP  Petróleo e Gás  2021-03-31   
4         3R PETROLEUM ÓLEO E GÁS S.A.   RRRP  Petróleo e Gás  2021-06-30   

DS_CONTA  Ativo Circulante  Ativo Total  Contas a Receber  \
0                     3043       347642                 0   
1                     1975       323655                 0   
2                     1280       320788                 0   
3                  1378821      1829110            790530   
4                  1161876      1841909                 0   

DS_CONTA  Lucro/Prejuízo do Período  Obrigações Fiscais  \
0                          -27951.0                  98   
1                          -42319.0                 

In [37]:
table2.to_csv(rf"Cuspidão\Completao.csv",decimal=',',sep=';', encoding='latin-1',index=False)